In [8]:
# Ячейка 1: Импорт библиотек
import pandas as pd
import numpy as np
%pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Ячейка 2: Загрузка данных
prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

In [4]:
# Ячейка 3: Предобработка financial_data

# Получаем все колонки с месяцами
month_columns = [col for col in financial_data.columns if any(month in col for month in [
    'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
    'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'
])]

# Функция для очистки числовых значений
def clean_numeric(value):
    if isinstance(value, str):
        # Удаляем все пробелы (включая неразрывные) и заменяем запятые на точки
        value = value.replace('\xa0', '').replace(' ', '').replace(',', '.')
        # Обрабатываем текстовые маркеры
        if value.lower() in ['стоп', 'вноль', 'вноль', 'end']:
            return np.nan
    return value

# Применяем очистку ко всем колонкам с месяцами
for col in month_columns:
    financial_data[col] = (
        financial_data[col]
        .apply(clean_numeric)
        .astype(float)
    )

# Группируем и преобразуем в длинный формат
financial_data_clean = financial_data.groupby(['id', 'Account'])[month_columns].sum().reset_index()
financial_data_clean = financial_data_clean.melt(
    id_vars=['id', 'Account'],
    value_vars=month_columns,
    var_name='month',
    value_name='revenue'
)
financial_data_clean.rename(columns={'Account': 'AM'}, inplace=True)

In [5]:
# Ячейка 4: Объединение данных
merged = pd.merge(
    prolongations,
    financial_data_clean,
    on=['id', 'AM'],
    how='left'
)

# Преобразуем месяц окончания проекта в формат "Месяц Год"
merged['end_month'] = merged['month_x']  # Уже в нужном формате

In [6]:
# Ячейка 5: Расчет коэффициентов пролонгации

# Функция для извлечения года и месяца
def extract_year_month(s):
    parts = s.split()
    return f"{parts[0]} {parts[1]}"

# Создаем словарь для порядка месяцев
months_order = [
    'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023',
    'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023'
]

def calculate_prolongation(df):
    metrics = []
    for end_month in months_order:
        # Фильтруем проекты, завершившиеся в текущем месяце
        projects = df[df['end_month'] == end_month]

        # Сумма отгрузки в последний месяц
        total_last_month = projects[projects['month_y'] == end_month]['revenue'].sum()

        # Коэффициент K1 (пролонгация в первый месяц)
        next_month_1 = months_order[(months_order.index(end_month) + 1)] if end_month != 'Декабрь 2023' else None
        if next_month_1:
            k1 = projects[projects['month_y'] == next_month_1]['revenue'].sum() / total_last_month if total_last_month > 0 else 0
        else:
            k1 = 0

        # Коэффициент K2 (пролонгация во второй месяц)
        next_month_2 = months_order[(months_order.index(end_month) + 2)] if end_month not in ['Ноябрь 2023', 'Декабрь 2023'] else None
        if next_month_2 and total_last_month > 0:
            remaining = total_last_month - projects[projects['month_y'] == next_month_1]['revenue'].sum()
            k2 = projects[projects['month_y'] == next_month_2]['revenue'].sum() / remaining if remaining > 0 else 0
        else:
            k2 = 0

        metrics.append({
            'end_month': end_month,
            'K1': round(k1, 2),
            'K2': round(k2, 2)
        })
    return pd.DataFrame(metrics)

# Расчет по менеджерам
managers_report = merged.groupby('AM').apply(calculate_prolongation).reset_index().drop(columns='level_1')

# Расчет по отделу
department_report = calculate_prolongation(merged)

/var/folders/28/ry94lb9x7g9dlmnxp72zr5rr0000gn/T/ipykernel_11761/700092053.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  managers_report = merged.groupby('AM').apply(calculate_prolongation).reset_index().drop(columns='level_1')


In [9]:
# Ячейка 6: Экспорт в Excel


# Проверка содержимого DataFrame
print("Managers Report:")
print(managers_report)

print("Department Report:")
print(department_report)

with pd.ExcelWriter('prolongation_report_final.xlsx') as writer:
    managers_report.to_excel(writer, sheet_name='Менеджеры', index=False)
    department_report.to_excel(writer, sheet_name='Отдел', index=False)

Managers Report:
                               AM      end_month  K1  K2
0    Васильев Артем Александрович    Январь 2023   0   0
1    Васильев Артем Александрович   Февраль 2023   0   0
2    Васильев Артем Александрович      Март 2023   0   0
3    Васильев Артем Александрович    Апрель 2023   0   0
4    Васильев Артем Александрович       Май 2023   0   0
..                            ...            ...  ..  ..
115                       без А/М    Август 2023   0   0
116                       без А/М  Сентябрь 2023   0   0
117                       без А/М   Октябрь 2023   0   0
118                       без А/М    Ноябрь 2023   0   0
119                       без А/М   Декабрь 2023   0   0

[120 rows x 4 columns]
Department Report:
        end_month  K1  K2
0     Январь 2023   0   0
1    Февраль 2023   0   0
2       Март 2023   0   0
3     Апрель 2023   0   0
4        Май 2023   0   0
5       Июнь 2023   0   0
6       Июль 2023   0   0
7     Август 2023   0   0
8   Сентябрь 2023   0 